In [323]:
!pip install slack_sdk
!pip install huggingface_hub[hf_xet]
!pip install transformers==4.50.0

In [325]:
import os
from slack_sdk import WebClient
slack_token = "my_slack_token"
client = WebClient(token=slack_token)
cursor=None

def collect_txt_data(channel_id):
    client.conversations_join(channel=channel_id)
    response = client.conversations_history(channel=channel_id)
    messages = response["messages"]
    
    while 'response_metadata' in response and 'next_cursor' in response['response_metadata']:
        cursor = response['response_metadata']['next_cursor']
        response = client.conversations_history(channel=channel_id, limit=200, cursor=cursor)
        messages.extend(response['messages'])

    texts = [msg["text"] for msg in messages if "text" in msg and "subtype" not in msg]
    users = [msg["user"] for msg in messages if "text" in msg and "subtype" not in msg]
    res = [users, texts]
    return res

In [327]:
def make_input_data(channel):
    max_length = 256
    data = []
    for i in range(1, len(channel[0])):
        if channel[0][i-1] == "U026Z4RQXM5" and channel[0][i] != "U026Z4RQXM5" and "<https:" not in channel[1][i-1] and "<https:" not in channel[1][i]:
            data.append((channel[1][i], channel[1][i-1]))

    return data

In [329]:
b4_channel = collect_txt_data(channel_id="C07B6J6U6CR")
gen_channel = collect_txt_data(channel_id="C027L2L0L5S")
rnd_channel = collect_txt_data(channel_id="C027ARC13DK")
book_channel = collect_txt_data(channel_id="C026Z4TNWF9")
dplearn_channel = collect_txt_data(channel_id="C034N2LMRKM")
els_channel = collect_txt_data(channel_id="C02D3MY7057")

In [331]:
b4_data = make_input_data(b4_channel)
gen_data = make_input_data(gen_channel)
rnd_data = make_input_data(rnd_channel)
book_data = make_input_data(book_channel)
dplearn_data = make_input_data(dplearn_channel)
els_data = make_input_data(dplearn_channel)

In [333]:
all_data= b4_data+gen_data+rnd_data+book_data+dplearn_data+els_data

In [335]:
all_data

[('あと20分位で行けそうかも', 'いまどこ？<@U07AQUCTVCM> くん'),
 ('あと10分ほどで着きます', '6515教室でやりますんで、あつまってね'),
 ('今日の自己紹介の担当が <@U07NC06MJQ6> <@U07AQUCTVCM> <@U07BGMKSFNU> なのですが1人しか来れそうにないです :innocent: ',
  'あら…'),
 ('すみません多分Lab.Talk間に合わないです', 'おっけー\n他のみんなはどう？'),
 ('自分も小田急線遅延で遅れます', 'うーん、30分くらい遅らせないと無理やね'),
 ('zoomで参加でもいいですか？', 'みんな全体的に遅れてるから、少し開始時間を遅らせよう'),
 ('南武線、小田急で運転見合わせしてるので学校に辿り着かなそうです', 'おだきゅうせーん:exploding_head:'),
 ('私用で今週の水曜は欠席します。', 'あと、春学期が始まったので、Lab. Talkと輪講は休まないようによろしくです'),
 ('<@U026Z4RQXM5> 来週からの自己紹介ってなにか準備した方がいいですか？', 'そっすね\n軽く2,3枚のスライドを作りましょうか'),
 ('まじすか！了解です！ありがとうございます！', '先々週で一段落したはず'),
 ('輪行はありますか...？', 'そっちもないよ'),
 ('来週からですか！！勘違いしてました！', '来週よろしゅう'),
 ('labtalk遅れそうです...すいません...', '今日はないよん'),
 ('4月にやるって言ってたんですけど詳しい日程は決まってなかったと思います',
  'おけ、今週はスルーしましょう\n来週からにしましょう\nリマインドありがとう'),
 ('lab talkでの自己紹介は来週から始まりますか？', 'いつだっけ…\n来週だっけ'),
 ('<@U026Z4RQXM5> マージリクエストを作成したので確認お願いします。', 'りょうかいです'),
 ('今週のlabtalkとゼミ、サークルのためお休みします。', 'おけ'),
 ('すみませんありがとうございます', 'そろそろ始めましょうかね'),
 ('教室どこですか？', '6508だよ'),
 ('同じ

In [337]:
import requests
import json

file_path = r'C:\Users\User\datasets\question_answers.json\question_answers.json'


# JSONファイルを読み込む
with open(file_path, 'r', encoding='utf-8') as file:
    data = json.load(file)

In [339]:
subset = data[:1000]
subdata = []

for k in range(1000):
    for i in range(8):
        subdata.append((subset[k]["qas"][i]["question"], subset[k]["qas"][i]["answer"]))

In [341]:
all = all_data + subdata
all

[('あと20分位で行けそうかも', 'いまどこ？<@U07AQUCTVCM> くん'),
 ('あと10分ほどで着きます', '6515教室でやりますんで、あつまってね'),
 ('今日の自己紹介の担当が <@U07NC06MJQ6> <@U07AQUCTVCM> <@U07BGMKSFNU> なのですが1人しか来れそうにないです :innocent: ',
  'あら…'),
 ('すみません多分Lab.Talk間に合わないです', 'おっけー\n他のみんなはどう？'),
 ('自分も小田急線遅延で遅れます', 'うーん、30分くらい遅らせないと無理やね'),
 ('zoomで参加でもいいですか？', 'みんな全体的に遅れてるから、少し開始時間を遅らせよう'),
 ('南武線、小田急で運転見合わせしてるので学校に辿り着かなそうです', 'おだきゅうせーん:exploding_head:'),
 ('私用で今週の水曜は欠席します。', 'あと、春学期が始まったので、Lab. Talkと輪講は休まないようによろしくです'),
 ('<@U026Z4RQXM5> 来週からの自己紹介ってなにか準備した方がいいですか？', 'そっすね\n軽く2,3枚のスライドを作りましょうか'),
 ('まじすか！了解です！ありがとうございます！', '先々週で一段落したはず'),
 ('輪行はありますか...？', 'そっちもないよ'),
 ('来週からですか！！勘違いしてました！', '来週よろしゅう'),
 ('labtalk遅れそうです...すいません...', '今日はないよん'),
 ('4月にやるって言ってたんですけど詳しい日程は決まってなかったと思います',
  'おけ、今週はスルーしましょう\n来週からにしましょう\nリマインドありがとう'),
 ('lab talkでの自己紹介は来週から始まりますか？', 'いつだっけ…\n来週だっけ'),
 ('<@U026Z4RQXM5> マージリクエストを作成したので確認お願いします。', 'りょうかいです'),
 ('今週のlabtalkとゼミ、サークルのためお休みします。', 'おけ'),
 ('すみませんありがとうございます', 'そろそろ始めましょうかね'),
 ('教室どこですか？', '6508だよ'),
 ('同じ

In [343]:
src = []
trg = []
for i in range(len(all)):
    src.append(all[i][0])
    trg.append(all[i][1])

In [345]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments

# トークナイザーとモデルのロード
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token
model = GPT2LMHeadModel.from_pretrained('gpt2')

# トークン化とエンコーディング
max_length = 132

src_encodings = tokenizer(src[100:180], padding='max_length', truncation=True, max_length=max_length, return_tensors="pt")
trg_encodings = tokenizer(trg[100:180], padding='max_length', truncation=True, max_length=max_length, return_tensors="pt")


In [347]:
tmp_src = {}
tmp_trg = {}
tmp_src["input_ids"] = src_encodings["input_ids"][:80]
tmp_src["attention_mask"] = src_encodings["attention_mask"][:80]
tmp_trg["input_ids"] = trg_encodings["input_ids"][:80]
tmp_trg["attention_mask"] = trg_encodings["attention_mask"][:80]

In [349]:
import torch
from torch.utils.data import Dataset

class CustomDataset(Dataset):
    def __init__(self, src_encodings, trg_encodings):
        self.src_encodings = src_encodings
        self.trg_encodings = trg_encodings

    def __getitem__(self, idx):
        item = {key: val[idx].clone().detach() for key, val in self.src_encodings.items()}
        item['labels'] = self.trg_encodings['input_ids'][idx].clone().detach()
        return item

    def __len__(self):
        return len(self.src_encodings['input_ids'])

dataset = CustomDataset(tmp_src, tmp_trg)


In [353]:
from transformers import GPT2LMHeadModel, Trainer, TrainingArguments

# モデルのロード
model = GPT2LMHeadModel.from_pretrained('gpt2')

# トレーニング設定
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=4,
    save_steps=10_000,
    save_total_limit=2,
)

# トレーナーの設定
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset
)

# トレーニングの実行
trainer.train()



Step,Training Loss


TrainOutput(global_step=60, training_loss=0.679086176554362, metrics={'train_runtime': 195.6445, 'train_samples_per_second': 1.227, 'train_steps_per_second': 0.307, 'total_flos': 16167444480000.0, 'train_loss': 0.679086176554362, 'epoch': 3.0})

In [363]:
# テストデータの定義
test_texts = ['ここに人は何人いますか', "この文字の色は何色ですか"]

# トークン化とエンコーディング
test_encodings = tokenizer(test_texts, padding="max_length", truncation=True, return_tensors="pt", max_length=max_length)

# モデルによる予測
model.eval()
with torch.no_grad():
    outputs = model(**test_encodings)
    predictions = outputs.logits.argmax(dim=-1)

# 予測結果のデコード
decoded_predictions = [tokenizer.decode(pred, skip_special_tokens=True) for pred in predictions]
print(decoded_predictions)


['�', '��']
